<a href="https://colab.research.google.com/github/Macia-Comella/03MAIR---Algoritmos-de-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Maci%C3%A0_Comella_Barb%C3%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Macià Comella Barbé  <br>
Url: https://github.com/Macia-Comella/03MAIR---Algoritmos-de-Optimizacion/blob/main/Trabajo_Pr%C3%A1ctico_Maci%C3%A0_Comella_Barb%C3%A9.ipynb <br>
Google Colab: https://colab.research.google.com/drive/1FGZOnWffgCmwZOaOAiDOFlKo6aX0BwOK?usp=sharing <br>
Problema:
>1. Sesiones de doblaje <br>

Descripción del problema:

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible. Los datos son:

Número de actores: 10  
Número de tomas : 30  
Actores/Tomas : https://bit.ly/36D8IuK

....







                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

### Respuesta

**Espacio de soluciones:**  
Representamos el espacio de soluciones con una lista de listas que contiene las sesiones programadas cada día. Cada lista representa un día y éste contiene las sesiones de dicho día. El tamaño de las listas deberá limitarse a 6 ya que no se pueden programar más de 6 tomas por día.

Asociada a esta lista de programación de tomas generamos una lista de listas con los actores que deben desplazarse al estudio cada día siguiendo la misma lógica (cada lista representa un día y contiene los actores necesarios para tal día)

Trabajamos con soluciones compactas intentando encabir el máximo de tomas en un día. Como el límite es 6 tomas/día y hay 30 tomas a programar se pueden compactar en 5 días. Se toma este enfoque ya que, en general, repartir las tomas a lo largo de más días incrementa la probabilidad de que actores deban desplazarse más días al estudio. Visto desde otro punto de vista, una vez hemos programado una toma en un día, los actores necesarios para esa toma ya están desplazados al estudio, por lo que todas las tomas adicionales que puedan grabar no añaden coste adicional.

In [1]:
import random
import math
from copy import deepcopy

In [2]:
# Carga datos de tomas y actores
planificacion = {'1': [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
                 '2': [0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
                 '3': [0, 1, 0, 0, 1, 0, 1, 0, 0, 0],
                 '4': [1, 1, 0, 0, 0, 0, 1, 1, 0, 0],
                 '5': [0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
                 '6': [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
                 '7': [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
                 '8': [1, 1, 0, 0, 0, 1, 0, 0, 0, 0],
                 '9': [1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
                 '10': [1, 1, 0, 0, 0, 1, 0, 0, 1, 0],
                 '11': [1, 1, 1, 0, 1, 0, 0, 1, 0, 0],
                 '12': [1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
                 '13': [1, 0, 0, 1, 1, 0, 0, 0, 0, 0],
                 '14': [1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
                 '15': [1, 1, 0, 0, 0, 0, 1, 0, 0, 0],
                 '16': [0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
                 '17': [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                 '18': [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
                 '19': [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                 '20': [1, 0, 1, 1, 1, 0, 0, 0, 0, 0],
                 '21': [0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
                 '22': [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                 '23': [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                 '24': [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
                 '25': [1, 1, 0, 1, 0, 0, 0, 0, 0, 1],
                 '26': [1, 0, 1, 0, 1, 0, 0, 0, 1, 0],
                 '27': [0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
                 '28': [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                 '29': [1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
                 '30': [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]}

# Transformación del formato de los actores para tener, para cada toma, el identificador del actor en lugar de una tabla binaria
for k in planificacion:
  lista = []
  for numero, actor in enumerate(planificacion[k]):
    if actor == 1:
      lista.append(numero+1)
  planificacion[k] = lista

print(planificacion)

{'1': [1, 2, 3, 4, 5], '2': [3, 4, 5], '3': [2, 5, 7], '4': [1, 2, 7, 8], '5': [2, 4, 8], '6': [1, 2, 4, 5], '7': [1, 2, 4, 5], '8': [1, 2, 6], '9': [1, 2, 4], '10': [1, 2, 6, 9], '11': [1, 2, 3, 5, 8], '12': [1, 2, 3, 4, 6], '13': [1, 4, 5], '14': [1, 3, 6], '15': [1, 2, 7], '16': [4, 10], '17': [1, 3], '18': [3, 6], '19': [1, 3], '20': [1, 3, 4, 5], '21': [6, 8], '22': [1, 2, 3, 4], '23': [1, 3], '24': [3, 6], '25': [1, 2, 4, 10], '26': [1, 3, 5, 9], '27': [4, 5], '28': [1, 4], '29': [1, 5, 6], '30': [1, 4]}


In [3]:
# Espacio de soluciones

def genera_programacion_aleatoria(): # Genera una solución aleatoria de 5 días
  programacion = [[]]
  tomas_a_programar = list(range(1,31))

  while len(tomas_a_programar) > 0:
    for dia in programacion:
      programado = False
      if len(dia) < 6:
        toma_programada = random.choice(tomas_a_programar)
        dia.append(toma_programada)
        tomas_a_programar.remove(toma_programada)
        programado = True

    if not programado:
      programacion.append([])

  return programacion

In [4]:
def programa_actores(programacion, plan): # En base a una solución genera los actores necesarios para cada día
  programacion_actores = []
  for dia, tomas_dia in enumerate(programacion):
    programacion_actores.append([])
    for toma in tomas_dia:
      for tomas_plan in plan[str(toma)]:
        programacion_actores[dia].append(tomas_plan)

  for dia in range(len(programacion_actores)):
    programacion_actores[dia] = set(programacion_actores[dia])

  return programacion_actores


In [5]:
programacion_aleatoria = genera_programacion_aleatoria()

prog_actores = programa_actores(programacion_aleatoria, planificacion)

dia = 1
for tomas, actores in zip(programacion_aleatoria, prog_actores):
  print(f"El día {dia} se programan las tomas {tomas}, por lo tanto se necesitan los actores {actores}")
  dia += 1

El día 1 se programan las tomas [12, 3, 23, 8, 27, 22], por lo tanto se necesitan los actores {1, 2, 3, 4, 5, 6, 7}
El día 2 se programan las tomas [10, 29, 2, 7, 11, 13], por lo tanto se necesitan los actores {1, 2, 3, 4, 5, 6, 8, 9}
El día 3 se programan las tomas [26, 9, 6, 28, 15, 14], por lo tanto se necesitan los actores {1, 2, 3, 4, 5, 6, 7, 9}
El día 4 se programan las tomas [19, 16, 24, 17, 30, 21], por lo tanto se necesitan los actores {1, 3, 4, 6, 8, 10}
El día 5 se programan las tomas [20, 4, 1, 5, 18, 25], por lo tanto se necesitan los actores {1, 2, 3, 4, 5, 6, 7, 8, 10}


**Función objetivo:**  
La función objetivo a minimizar es el coste del desplazamiento de los actores al estudio de grabación. Definimos el coste de convocar a un actor al estudio de grabación con valor 1 por cada día que un actor deba desplazarse. Con esta definición, el coste a minimizar es el número de actores convocados cada día.  

A continuación definimos la función coste y calculamos el coste de la solución aleatoria generada en el apartado anterior

In [6]:
def coste(prog_actores): # En base a una programación de actores calcula el coste en unidades monetarias
  coste = 0
  for dia in prog_actores:
    coste += len(dia)

  return coste

In [7]:
coste_prog_aleatoria = coste(prog_actores)
print(f"El coste de la programación aleatoria es de {coste_prog_aleatoria} unidades")

El coste de la programación aleatoria es de 38 unidades


**Restricciones:**  
El enunciado del problema nos impone dos restricciones:
1. No pueden programarse más de 6 tomas cada día.
2. Deben programarse las 30 tomas.

La primera restricción la implementamos en todas las funciones que generen soluciones o modificaciones de las soluciones, la longitud de la lista de soluciones programadas por día no puede ser superior a 6.  

La segunda restricción la implementamos asegurando que las soluciones incluyen el total de las 30 tomas a programar. Más adelante se crea una función que comprueba la factibilidad de una programación asegurando éste hecho.

Adicionalmente, el enunciado nos proporciona una tabla que condiciona los actores que deben desplazarse al estudio cada día en función de las tomas que están programadas ese día. Esta condición en realidad no impone ninguna restricción, solo influirá en el valor de la función objetivo.

#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

### Respuesta
El espacio de soluciones completo incluye todas las posibles combinaciones de tomas programadas por día, teniendo en cuenta que se programan 6 tomas por día para un total de 5 días.  

Para el cómputo del espacio de soluciones consideramos que si las tomas se programan en diferente orden dentro de un mismo día se consideran soluciones diferentes, aunque tengan el mismo valor de coste. Lo mismo ocurre si intercambiamos el orden de días. Siguiendo esta lógica el espacio de soluciones se contabiliza del siguiente modo:

1. El 1r día hay 30 tomas a programar, de las cuales se programan 6. No se puede repetir pero el orden de las tomas importa, por lo tanto estamos ante un caso de variación sin repetición con fórmula: 30!/(30-6)! = 30!/24! = 30 * 29 * 28 * 27 * 26 * 25
2. El 2o día quedan 24 tomas a programar. Aplicamos la misma función actualizada para las tomas restantes: 24!/(24-6)! = 24!/18! = 24 * 23 * 22 * 21 * 20 * 19

Siguiendo este proceso hasta programar las 30 tomas el cómputo total de soluciones es:  
30!/24! * 24!/18! * 18!/12! * 12!/6! * 6! = 30!

Generalizando la solución, con las mismas restricciones, el orden de complejidad es factorial, O(n) = n!

In [29]:
# A continuación se calcula el valor concreto de soluciones para n = 30 -> factorial de 30
print(f"El espacio de soluciones contiene {math.factorial(30)} soluciones!")

El espacio de soluciones contiene 265252859812191058636308480000000 soluciones!


#Diseño
- ¿Que técnica utilizo? ¿Por qué?

### Propuesta 1: algoritmo hibrido

En una primera aproximación utilizaremos una combinación de búsqueda local y multiarranque para mejorar una solución aleatoria inicial. El multi-arranque nos aporta diversificación al empezar en varios puntos diferentes, mientras que la búsqueda local nos aporta intensificación en estos puntos iniciales aleatorios. Como la búsqueda local encuentra soluciones bastante rápido  podemos hacer bastantes arranques para empezar en sitios diferente.  

En una revisión del código, añadimos una componente de la búsqueda tabú: como la búsqueda local siempre encuentra la misma solución dada una solución inicial, restringimos las soluciones aleatorias iniciales a que no se repitan con las soluciones generadas anteriormente. De esta forma aseguramos que la generación aleatoria no nos lleva a invertir recursos en caminos ya explorados

In [ ]:
# Propuesta 1: combinar multi-arranque con búsqueda local
def genera_vecinos_2opt(programacion, plan): # Para una solución dada genera todas las posibles variaciones intercambiando dos tomas
  mejor_solucion = programacion
  prog_actores = programa_actores(programacion, plan)
  mejor_coste = coste(prog_actores) # Usamos la programación actual como valor a comparar. Si la generada no es mejor que la actual no queremos cambiar

  vecina = deepcopy(programacion)
  for dia in range(len(programacion)-1):
    for toma in range(len(programacion[dia])):
      for toma_dia_cambio in range(len(programacion[dia])):

        vecina[dia][toma] = programacion[dia+1][toma_dia_cambio]
        vecina[dia+1][toma_dia_cambio] = programacion[dia][toma]

        prog_actores_vec = programa_actores(vecina, plan)
        coste_vec = coste(prog_actores_vec)

        if coste_vec < mejor_coste:
          mejor_solucion = vecina
          mejor_coste = coste_vec

        vecina = deepcopy(programacion)

  return mejor_solucion

# Generamos la función completa. Ésta genera una programación aleatoria inicial y aplica la función genera_vecinos_2opt
# hasta que no somos capaces de mejorar la solución.
# No permite que se repitan programaciones aleatorias iniciales.
def multiarranque_busq_local(plan, intentos):
  mejor_sol_global = []
  mejor_coste_global = float('inf')
  sol_ya_intentadas = []

  for _ in range(intentos):
    parada = False

    sol_propuesta = genera_programacion_aleatoria()

    while sol_propuesta in sol_ya_intentadas:
      sol_propuesta = genera_programacion_aleatoria()

    mejor_sol_local = sol_propuesta
    mejor_coste_local = float('inf')
    sol_ya_intentadas.append(sol_propuesta)

    while parada == False:
      sol_busq_local = genera_vecinos_2opt(mejor_sol_local, plan)
      prog_actores_busq_local = programa_actores(sol_busq_local, plan)
      coste_sol_busq_local = coste(prog_actores_busq_local)

      if coste_sol_busq_local < mejor_coste_local:
        mejor_sol_local = sol_busq_local
        mejor_coste_local = coste_sol_busq_local

      else:
        parada = True

    if mejor_coste_local < mejor_coste_global:
      mejor_sol_global = mejor_sol_local
      mejor_coste_global = mejor_coste_local

  return mejor_sol_global

In [ ]:
print(f"Programación aleatoria: {programacion_aleatoria}")
print(f"Coste programación aleatoria: {coste_prog_aleatoria}")

sol_multi_busq_local = multiarranque_busq_local(planificacion, intentos=500)
print(f"Programación multiarranque con búsqueda local : {sol_multi_busq_local}")

prog_actores_multi_busq_local = programa_actores(sol_multi_busq_local, planificacion)
coste_multi_busq_local = coste(prog_actores_multi_busq_local)

print(f"Coste de la programación multiarranque con búsqueda local : {coste_multi_busq_local}")

# Habiendo lanzado varias ejecuciones, con 500 intentos vemos que la mejor solución a
# la que podemos llegar con esta heurística está en el entorno de 29 unidades de coste

Programación aleatoria: [[18, 20, 14, 3, 29, 11], [30, 22, 21, 25, 26, 13], [5, 27, 8, 19, 28, 10], [16, 12, 4, 7, 2, 24], [9, 17, 1, 23, 6, 15]]
Coste programación aleatoria: 40
Programación multiarranque con búsqueda local : [[20, 26, 2, 13, 30, 27], [24, 14, 21, 19, 29, 17], [4, 5, 1, 11, 23, 15], [10, 22, 8, 18, 28, 12], [6, 16, 7, 9, 25, 3]]
Coste de la programación multiarranque con búsqueda local : 29


In [ ]:
# Como sabemos que no podemos repetir caminos ya explorados y el tiempo de ejecucion
# con el hardware de Colab no es grande lanzamos una ejecución con 2000 intentos para
# obtener una solución bastante buena con esta heurística

sol_multi_busq_local = multiarranque_busq_local(planificacion, intentos=2000)
print(sol_multi_busq_local)

prog_actores_multi_busq_local = programa_actores(sol_multi_busq_local, planificacion)
coste_multi_busq_local = coste(prog_actores_multi_busq_local)

print(coste_multi_busq_local)

# El tiempo de ejecución ha sido de 1 minuto

[[10, 6, 8, 12, 9, 26], [1, 7, 16, 20, 2, 25], [23, 24, 18, 19, 14, 17], [27, 29, 28, 13, 30, 21], [5, 22, 15, 11, 3, 4]]
28


### Propuesta 2: algoritmo genético

A continuación programamos un algoritmo genético para resolver la misma instancia del problema de programación de actores. El objetivo es que mediante la mayor complejidad de los algoritmos genéticos y su equilibrio entre intensifiación y diversificación consigamos una mejor solución que el algoritmo anterior.

In [8]:
# Definimos la función de generar una población de N indivíduos
# Reaprovechamos la función genera_programacion_aleatoria del apartado anterior
def Generar_Poblacion(N):
  poblacion = []
  for _ in range(N):
    poblacion.append(genera_programacion_aleatoria())
  return poblacion

In [9]:
def Evaluar_Programacion(programa, plan): # Evaluamos la mejor programación de la población
  prog_actores = programa_actores(programa, plan)
  c = coste(prog_actores)
  return c

In [10]:
def Evaluar_Mejor_Programacion(poblacion, plan): # Evaluamos la mejor programación de la población
  mejor_programacion = []
  mejor_coste = float('inf')
  for programa in poblacion:
    c = Evaluar_Programacion(programa, plan)
    if c < mejor_coste:
      mejor_programacion = programa
      mejor_coste = c

  return mejor_programacion

In [11]:
def Factible(programa): # Función para asegurar que una programación es factible
  lista = []
  for dia in range(len(programa)):
    lista += programa[dia]
  return len(set(lista)) == 30

In [12]:
# Función para generar 2 hijos en base a 2 programaciones existentes.
# Se recombinan los alelos de cada padre aleatoriamente entre los hijos
# Llama a la función "factible" para asegurar que los hijos sean válidos
# FUNCIÓN NO VÁLIDA PORQUÉ REQUIERE UN TIEMPO DEMASIADO ALTO PARA GENERAR 2 HIJOS FACTIBLES CON ESTE NIVEL DE ALEATORIEDAD

def Genera_Hijos(programa1, programa2):
  parar = False
  while parar == False:
    hijo1_temp = []
    hijo2_temp = []
    hijo1 = []
    hijo2 = []
    for dia_n in range(len(programa1)):
      for toma_n in range(len(programa1[dia_n])):
        p = random.random()
        if p < 0.5:
          hijo1_temp.append(programa1[dia_n][toma_n])
          hijo2_temp.append(programa2[dia_n][toma_n])
        else:
          hijo1_temp.append(programa2[dia_n][toma_n])
          hijo2_temp.append(programa1[dia_n][toma_n])
    for i in [0,6,12,18,24]: # La función anterior genera una lista de 30 tomas. Las agrupamos en 5 días
      hijo1.append(hijo1_temp[i:i+6])
      hijo2.append(hijo1_temp[i:i+6])
    parar = Factible(hijo1) and Factible(hijo2)

  return [hijo1, hijo2]

In [13]:
# Función más simple que la anterior y más rápida de ejecutar
# Función para generar 2 hijos en base a 2 programaciones existentes.
# Se hace una partición por un día aleatoriamente (de los 5 días que tienen todas las programaciones)
# y se combinan padre y madre. Llama a la función "factible" para asegurar que los hijos sean válidos

def Genera_Hijos(programa1, programa2):
  parar = False
  while parar == False:
    particion = random.randint(0,5)
    hijo1 = programa1[0:particion] + programa2[particion:5]
    hijo2 = programa2[0:particion] + programa1[particion:5]
    parar = Factible(hijo1) and Factible(hijo2) # Paramos cuando ambos hijos son factibles

  return [hijo1, hijo2]

In [14]:
# Función de mutación de una programación. Escoge aleatoriamente 2 alelos y los intercambia
# Llama a la función Factible para asegurar que la programación es válida
def Mutar(programacion):
  parar = False
  while parar == False:
    programacion_mutada = deepcopy(programacion)
    gen1 = [random.randint(0,4), random.randint(0,5)]
    gen2 = [random.randint(0,4), random.randint(0,5)]
    programacion_mutada[gen1[0]][gen1[1]] = programacion[gen2[0]][gen2[1]]
    programacion_mutada[gen2[0]][gen2[1]] = programacion[gen1[0]][gen1[1]]
    parar = Factible(programacion_mutada) # Paramos cuando la mutación ha generado una programación factible

  return programacion_mutada

In [15]:
# Recibe una población y una planificación de tomas y genera una población extendida con hijos
# Se duplica la población mediante la función Genera_Hijos escogiendo a los padres según su fitness
# Se mutan el 50% de los hijos aleatoriamente
# Se selecciona la población hasta dejarla en la dimensión inicial. El percentil superior 25% se selecciona directamente
# El percentil 25% siguiente se selecciona por selección ponderada por la inversa del coste
# El percentil inferior 50% se descarta para mantener el tamaño de población constante
def Cruzar_Seleccionar(poblacion, plan):
  nueva_poblacion = deepcopy(poblacion)
  for _ in range(len(poblacion)//2):
    poblacion_copy = deepcopy(poblacion)
    madre = random.choice(poblacion_copy) # Escogemos madre aleatoriamente
    poblacion_copy.remove(madre)
    padre = random.choice(poblacion_copy) # Escogemos padre aleatoriamente
    hijos_generados = Genera_Hijos(madre, padre) #Generamos 2 hijos

    for i in range(1):
      if random.random() > 0.5:
        hijos_generados[i] = Mutar(hijos_generados[i]) # Mutamos hijo el 50% de las veces

    nueva_poblacion.append(hijos_generados[0])
    nueva_poblacion.append(hijos_generados[1])

  costes_inv = []
  lista_programaciones = []
  for programa in nueva_poblacion:
    cost = Evaluar_Programacion(programa, plan)
    lista_programaciones.append([cost, programa])
    costes_inv.append(1/cost)
  lista_programaciones.sort(key=lambda l: l[0])
  costes_inv.sort(reverse=True)

  nueva_poblacion_final = []
  for _ in range(len(poblacion)//2): # Añadimos el top 25% de la nueva población ampliada a la poblacion final (elitismo)
    nueva_poblacion_final.append(lista_programaciones[0][1])
    lista_programaciones = lista_programaciones[1:]
    costes_inv = costes_inv[1:]

  for _ in range(len(poblacion)//2): # Añadimos el siguiente 25% por selección ponderada por la inversa del coste
    programa_escogido = random.choices(lista_programaciones, weights=costes_inv)
    nueva_poblacion_final.append(programa_escogido[0][1])
    lista_programaciones.remove(programa_escogido[0])
    costes_inv.remove(1/programa_escogido[0][0])

  return nueva_poblacion_final

In [16]:
def Algoritmo_Genetico(plan, N, generaciones):
  poblacion = Generar_Poblacion(N)

  for _ in range(generaciones):
    poblacion = Cruzar_Seleccionar(poblacion, plan)

  programacion_final = Evaluar_Mejor_Programacion(poblacion, plan)
  c = Evaluar_Programacion(programacion_final, plan)
  return c, programacion_final

In [26]:
# Lanzamiento del algoritmo con una población "pequeña" y pocas generaciones para una ejecución rápida
%time c, programa = Algoritmo_Genetico(planificacion, N=100, generaciones=100)
print(f"La mejor programacion tiene un coste de {c} unidades")
print(f"¿Se han programado todas las tomas? {Factible(programa)}")
print("La mejor programación consiste en:")
for i in range(len(programa)):
  print(f"Tomas dia {i+1}: {programa[i]}")
# Obtenemos un resultado bastante bueno con un coste de 30 unidades

CPU times: user 12.6 s, sys: 32.5 ms, total: 12.7 s
Wall time: 12.8 s
La mejor programacion tiene un coste de 31 unidades
¿Se han programado todas las tomas? True
La mejor programación consiste en:
Tomas dia 1: [7, 18, 10, 2, 14, 26]
Tomas dia 2: [16, 27, 23, 25, 1, 13]
Tomas dia 3: [5, 9, 28, 22, 17, 30]
Tomas dia 4: [29, 19, 6, 20, 12, 24]
Tomas dia 5: [15, 21, 11, 4, 3, 8]


In [32]:
# Lanzamiento del algoritmo con una población "grande" y bastantes generaciones para una ejecución más cercana al óptimo
%time c, programa = Algoritmo_Genetico(planificacion, N=400, generaciones=200)
print(f"La mejor programacion tiene un coste de {c} unidades")
print(f"¿Se han programado todas las tomas? {Factible(programa)}")
print("La mejor programación consiste en:")
for i in range(len(programa)):
  print(f"Tomas dia {i+1}: {programa[i]}")

CPU times: user 8min 15s, sys: 1.4 s, total: 8min 17s
Wall time: 8min 21s
La mejor programacion tiene un coste de 28 unidades
¿Se han programado todas las tomas? True
La mejor programación consiste en:
Tomas dia 1: [23, 12, 24, 8, 18, 19]
Tomas dia 2: [15, 25, 16, 4, 30, 5]
Tomas dia 3: [28, 9, 7, 6, 27, 13]
Tomas dia 4: [11, 10, 29, 26, 14, 21]
Tomas dia 5: [2, 3, 17, 22, 20, 1]


### Conclusión:
Con una ejecución larga del algoritmo genético sólo conseguimos resultados de una calidad igual a la del algoritmo híbrido de la propuesta 1. Para mejorar este resultado se podrían modificar las funciones de cruce y/o mutación para experimentar con otros métodos. También se podría incrementar el tamaño de la población o las generaciones a costa de incrementar el tiempo de ejecución.